In [1]:
import json
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sys import getsizeof
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/felicityzhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
inverted_index = {}
STOP_WORDS = set(stopwords.words('english'))

In [4]:
def lemma(word):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

In [ ]:
def remove_u(word):
    word_u = (word.encode('unicode-escape')).decode("utf-8", "strict")
    if r'\u' in word_u: 
        return word_u.split('\\u')[1]
    return word

In [ ]:
#Read json file
news_json = open("title_links.json")
news = json.load(news_json)
for item in news:
    k,v = item["title"], item["link"]
    url = v
    html = urlopen(url)
    soup = BeautifulSoup(html)
    k = remove_u(k)
    
    #Fetch Contents
    contents = soup.find_all('p')
    article = []
    for c in contents:
        elem = c.get_text(strip=True)
        elem = elem.replace('\xa0', ' ')
        if elem != '':
            article.append(elem)
    
    #all_text to lower case, split by nonalphanumeric characters
    for text in article:
        text = re.sub(r'[^a-z0-9]', ' ', text.lower())
        
        for word in text.split():
            if len(word) > 2:
                word=lemma(word)
                if word not in STOP_WORDS:
                    if word not in inverted_index:
                        word_dict = {k:1}
                        inverted_index[word] = word_dict
                    else:
                        if k in inverted_index[word]:
                            inverted_index[word][k] += 1
                        else:
                            word_dict = {k:1}
                            inverted_index[word].update(word_dict)


In [ ]:
with open('title_links.json') as f:
    title_link = json.load(f)

In [ ]:
#sort frequencies for each key words
all_queries = []
final_list = []
for k,v in inverted_index.items():
    sorted_tuples = sorted(v.items(), key=lambda item: -item[1])
    for elem in sorted_tuples:
        temp_dict = {}
        temp_dict["counts"] = elem[1]
        temp_dict["query"] = k
        if k not in all_queries:
            all_queries.append(k)
        temp_dict["title"] = elem[0]
        for i in title_link:
            if i['title'] == elem[0]:
                temp_dict["link"] = i['link']
        
        #avoid a dict if missing keys
        if "link" in temp_dict and "title" in temp_dict and "query" in temp_dict and "counts" in temp_dict: 
            final_list.append(temp_dict)    

In [ ]:
print(len(final_list))
print(len(all_queries))

In [ ]:
with open("inverted_index_words.json", "w") as outfile:  
    json.dump(final_list, outfile)